In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import pathlib

p = pathlib.Path().resolve()

while True:
    if (p / ".git").exists():
        os.chdir(p)
        print("프로젝트 루트:", p)
        break

    if p.parent == p:
        raise RuntimeError("프로젝트 루트를 찾지 못함.")

    p = p.parent

프로젝트 루트: E:\AI_Project\codeit_ai_rfp_rag_assistant


In [3]:
from src.document_loader import get_hwp_text
test_file = "./Data/files/국민연금공단_2024년 이러닝시스템 운영 용역.hwp"

test = get_hwp_text(test_file)
print(test[:500])

e:\AI_Project\codeit_ai_rfp_rag_assistant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2024년 이러닝시스템 운영 용역 제안요청서

2024년 이러닝시스템 운영 용역 제안요청서
 

청렴한 업무처리 및 부조리 신고
국민연금공단은 모든 계약업무를 투명하고 공정하게 처리하고 있습니다.
계약과정 및 계약이행 시 금품·향응·편의 제공을 요구하는 직원이 있거나, 지연·학연 등의 사유로 특혜를 제공하는 것을 보거나 들은 경우에는 공단 홈페이지 부조리신고센터로 신고하여 주시기 바랍니다.


부조리
신고센터
국민연금 홈페이지(www.nps.or.kr)  클린센터

레드휘슬(www.redwhistle.org)익명성 보장
2024. 1.

인사혁신실 인재개발부


목   차


 . 사업 안내	   	1
   1. 사업개요	   	1
   2. 사업목적	   	1
   3. 사업범위	   	1
   4. 기대효과	   	2
 . 제안요청 내용	   	3
   1. 사업 운영방향	   	3
  


In [4]:
import os
from src.chunking import load_chunks, save_chunks, load_and_chunk_all_docs
from src.vectorstore import build_vector_store

CHUNK_PATH = "./Data/chunks.pkl"
FILES_PATH = "./Data"

# ---------------------------------------------------------
# 1) 청크 존재 여부 확인
# ---------------------------------------------------------
if os.path.exists(CHUNK_PATH):
    print("기존 chunks.pkl 발견 — 로드합니다.")
    chunks = load_chunks(CHUNK_PATH)
else:
    print("기존 chunks.pkl 없음 — 새로 생성합니다.")
    chunks = load_and_chunk_all_docs(FILES_PATH)
    save_chunks(chunks, CHUNK_PATH)
    print("새 chunks.pkl 저장 완료")

# ---------------------------------------------------------
# 2) 벡터스토어 생성 또는 로드
# ---------------------------------------------------------
vector_store = build_vector_store(chunks)

# ---------------------------------------------------------
# 3) retriever 생성
# ---------------------------------------------------------
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

print("청크 & 벡터스토어 준비완료")

기존 chunks.pkl 발견 — 로드합니다.
📌 기존 벡터스토어 감지됨 — 새로 추가하지 않고 로드만 진행
청크 & 벡터스토어 준비완료


In [5]:
for i, doc in enumerate(chunks[:3]):
    print(f"\n[{i+1}] 문서명: {doc.metadata.get('source', '미상')}")
    print(doc.metadata)


[1] 문서명: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp
{'source': '(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp', 'org': '(사)벤처기업협회', 'category': '2024년 벤처확인종합관리시스템 기능 고도화 용역사업 입찰공고', 'doc_type': '제안요청서', 'budget': '352000000.0', 'open_date': '2024-03-19 10:49:46', 'start_date': 'nan', 'end_date': '2024-04-09 15:00:00', 'chunk_index': 0, 'chunk_id': '(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp_0'}

[2] 문서명: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp
{'source': '(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .hwp', 'org': '(사)벤처기업협회', 'category': '2024년 벤처확인종합관리시스템 기능 고도화 용역사업 입찰공고', 'doc_type': '제안요청서', 'budget': '352000000.0', 'open_date': '2024-03-19 10:49:46', 'start_date': 'nan', 'end_date': '2024-04-09 15:00:00', 'chunk_index': 1, 'chunk_id': '(사)벤처기업협회_2024년 베

In [6]:
docs = vector_store.get(include=["metadatas", "documents"])
print(len(docs["documents"]))

30776


In [7]:
import unicodedata

target_keyword = "기초과학연구원"  # 찾고 싶은 키워드

docs = vector_store.get(include=["metadatas", "documents"], limit=99999)
found = 0

for i, (meta, content) in enumerate(zip(docs["metadatas"], docs["documents"])):
    src = unicodedata.normalize("NFC", str(meta.get("source", ""))).strip()

    # '이러닝' 또는 자모분리형 '이러닝' 모두 검색
    if target_keyword in src or "기초과학연구원" in src:
        print(f"🧾 {i}번 문서명: {src}")
        print(f"본문 길이: {len(content)}자")
        print(f"앞 500자 미리보기:\n{content[:500]}")
        print("="*80)
        found += 1

🧾 8086번 문서명: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
본문 길이: 269자
앞 500자 미리보기:
[PAGE:1]
2025년도 중이온가속기용 극저온시스템
운전 용역 과업지시서
2024. 10.
중이온가속기연구소
가속기운영부 극저온팀

[PAGE:2]
문서번호
-
개정번호
0
발 행 일
2024. 10. 30
2025년도 중이온가속기용 극저온시스템
운전 용역 과업지시서
문서상태 식별 (다음 중 하나에 체크)
□ 문서상태-1 : 최종설계, 해석, 구매 등에 이용되는 설계 확인된 정보
□ 문서상태-2 : 추정기법 등 설계가정을 사용하여 기술된 정보
■ 문서상태-3 : 설계확인 및
🧾 8087번 문서명: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
본문 길이: 283자
앞 500자 미리보기:
여 기술된 정보
■ 문서상태-3 : 설계확인 및 추정되지 않은 정보
소속 및 직위
성 명
서 명
작 성
극저온팀/연구위원
유정현
검 토
품질관리실/연구위원
신일경
극저온팀/팀장
신재희
승 인
가속기운영부/부장
정연세

[PAGE:3]
2025년도 중이온가속기용 극저온시스템
운전 용역 과업지시서
문서번호 :
-
개정번호 :
0
발 행 일 :
2024. 10. 30
페 이 지 :
3/19
□ 개정이력
개정번호
개정일자
개  정  사  유
1
2024. 07. 03
최초발행

[PAGE:4]
2025년도 �
🧾 8088번 문서명: 기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf
본문 길이: 1150자
앞 500자 미리보기:
��  사  유
1
2024. 07. 03
최초발행

[PAGE:4]
2025년도 중이온가속기용 극저온시스템
운전 용역 과업지시서
문서번호 :
-
개정번호 :
0
발 행 일 :
2024. 10. 30
페 이 지 :
4/19
목       차
1. 서  론 ....................................................................

In [8]:
vs = vector_store.get(include=["documents", "metadatas"], limit=999999)

for meta, doc in zip(vs["metadatas"], vs["documents"]):
    if "기초과학연구원" in meta.get("org", ""):
        print("길이:", len(doc.strip()))
        print(doc[:200])
        print("----")

길이: 269
[PAGE:1]
2025년도 중이온가속기용 극저온시스템
운전 용역 과업지시서
2024. 10.
중이온가속기연구소
가속기운영부 극저온팀

[PAGE:2]
문서번호
-
개정번호
0
발 행 일
2024. 10. 30
2025년도 중이온가속기용 극저온시스템
운전 용역 과업지시서
문서상태 식별 (다음 중 하나에 체크)
□ 문서상태-1 : 최종설계, 해석, 구매 등에
----
길이: 283
여 기술된 정보
■ 문서상태-3 : 설계확인 및 추정되지 않은 정보
소속 및 직위
성 명
서 명
작 성
극저온팀/연구위원
유정현
검 토
품질관리실/연구위원
신일경
극저온팀/팀장
신재희
승 인
가속기운영부/부장
정연세

[PAGE:3]
2025년도 중이온가속기용 극저온시스템
운전 용역 과업지시서
문서번호 :
-
개정번호 :
0
발 행 일 :
2024. 10.
----
길이: 1149
��  사  유
1
2024. 07. 03
최초발행

[PAGE:4]
2025년도 중이온가속기용 극저온시스템
운전 용역 과업지시서
문서번호 :
-
개정번호 :
0
발 행 일 :
2024. 10. 30
페 이 지 :
4/19
목       차
1. 서  론 .........................................................
----
길이: 263
��서
문서번호 :
-
개정번호 :
0
발 행 일 :
2024. 10. 30
페 이 지 :
5/19
1. 서  론
1.1 목  적
  기초과학연구원 중이온가속기연구소(이하 “연구원”)에서 운영중인 중이온가속기용 극저
온시스템(이하 “극저온시스템”)은 고도의 공정 시스템으로, 안전하고 안정적인 중이온가속
기의 운영을 위해 상시 모니터링이 필요하다. 이에 본 
----
길이: 251
한다.
1.2 용어 설명
본 과업지시서에서 사용하는 용어에 대한 상세 내용은 표 1.과 같다. 계약자는 반드시 해당
용어의 의미를 충분히 이해하고 숙지해야 하며, 용어 미숙지로 인한 책임은 계약자에게 있
다.
용어
설명
연구원
중이온가속기연구

In [ ]:
import wandb

wandb.init(
    project="rfp-rag",     # 프로젝트 이름
    name="distilled-test", # 개별 런 이름
    config={
        "retriever": "Chroma + CrossEncoder",
        "llm_model": "gpt-5-mini",
        "distill": False
    }
)

In [9]:
from src.rag_graph import run_rag_graph

question = input()

result = run_rag_graph(question, vector_store, retriever)
print(result)

입력 질문: 고려대학교 차세대 포털 시스템 사업이랑 광주과학기술원의 학사 시스템 기능개선 사업을 비교해 줄래?
[COMPARE_JUDGE] type=비교, compare_keys=['고려대학교 차세대 포털 시스템 사업', '광주과학기술원의 학사 시스템 기능개선 사업']

[RETRIEVE] 멀티 리트리버 실행

[RERANK RESULTS]
 1. score=0.9999 | 고려대학교 | 차세대 포털·학사 정보시스템 구축사업 | 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf |  사업명: 고려대학교 차세대 포털·학사 정보시스템 구축 사업 나. 사업기간: 계약일로부터 24개월 이내 다. 무상유지보수기간 : 사업종료일로부터...
 2. score=0.9999 | 고려대학교 | 차세대 포털·학사 정보시스템 구축사업 | 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf | 략목표 달성 지원) 노후화된 시스템의 차세대 구축을 통 해 정보서비스의 품질을 강화하고 대학 교육 시스템의 경쟁력을 확보하고자 함  [PAGE:...
 3. score=0.9997 | 고려대학교 | 차세대 포털·학사 정보시스템 구축사업 | 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf | [PAGE:1] -1-  제 안 요 청 서  고려대학교 차세대 포털·학사 정보시스템 구축 사업  2024. 7. 01  ※ 본 자료는 제안내용의...
 4. score=0.9996 | 고려대학교 | 차세대 포털·학사 정보시스템 구축사업 | 고려대학교_차세대 포털·학사 정보시스템 구축사업.pdf | ··················································  236  Ⅵ. 제안서 작성 방법 ··········...
 5. score=0.9992 | 고려대학교 | 차

In [ ]:
question = input()

result = run_rag_graph(question, vector_store, retriever)
print(result)

In [ ]:
question = input()

result = run_rag_graph(question, vector_store, retriever)
print(result)

In [ ]:
question = input()

result = run_rag_graph(question, vector_store, retriever)
print(result)

입력 질문: 경희대학교에서 추진하고 있는 사업이 있어?
[COMPARE_JUDGE] type=단일, compare_keys=[]

[RETRIEVE] 멀티 리트리버 실행

[RERANK RESULTS]
 1. score=0.2011 | �� 대학에서 추진하는 사업 입찰과 관련하여 아래와 같이 제반사항을 준 수할 것을 서약합니다.  - 아    래 -     가. 제출된 모든 관...
 2. score=0.0407 | 경희대학교 및 산학협력단 요청 시 개인정보보호 강화를 위한 개선작업을 시행해야 함   - 개인정보를 취급하는 프로그램 운영 및 유지관리, 신규 ...
 3. score=0.0265 | 이터 분석 결과에 대한 다양한 시각화 도구 적용 필요   ❍ 학교 내 기관/조직에서 수집/저장/관리하고 있는 학생 관련 디지털 데이터 생성 시스...
 4. score=0.0228 | ��학연수생 관련 수작업 개선 ③ 외국인 학생/교원 맞춤형 영어서비스 지원 ④ 한국어과정 시스템 고도화     ❏ 국제교류협약관리 / 교류프로그...
 5. score=0.0169 | �� 제출하여야 함      경희대학교 및 산학협력단은 누출금지정보를 아래와 같이 지정함    - 경희대학교 소유 전산시스템의 내ㆍ외부 IP주소...
[JUDGE RAW OUTPUT]
 {"score":0.0,"reason":"부정확함 — 답변은 문서·메타데이터를 일부 반영(고려대학교의 '차세대 포털·학사 정보시스템 구축사업' 예산·일정 등)은 했으나, 문서와 메타데이터에 명시된 경희대학교 관련 자료를 무시하고 ‘경희대학교에서 추진하는 사업 언급 없음’이라고 잘못 결론지었습니다. 제공된 메타데이터(예: org: 경희대학교, category: [입찰공고] 산학협력단 정보시스템 운영 용역업체 선정, open_date: 2024-05-02)와 본문(개인정보보호·산학협력단 관련 문구)은 경희대학교 관련 사업·입찰 공고가 존재함을 명확히 보여주는데 이를 반영하지 않았습니다. 수치(예산·일정)는 문서 기반으로 제공되

In [ ]:
question = input()

result = run_rag_graph(question, vector_store, retriever)
print(result)

In [ ]:
wandb.finish()